Install exactcover from github


In [0]:
!pip install git+git://github.com/seemee/exactcover.git#egg=exactcover

  Cloning git://github.com/seemee/exactcover.git to /tmp/pip-install-nlf58lz7/exactcover
  Running command git clone -q git://github.com/seemee/exactcover.git /tmp/pip-install-nlf58lz7/exactcover
  Created wheel for exactcover: filename=ExactCover-0.2-cp36-cp36m-linux_x86_64.whl size=20791 sha256=e96b708c6fc7305e4591de1a80667900f59ac006dadf83bf4dd07e4e90e46b34
  Stored in directory: /tmp/pip-ephem-wheel-cache-hqau1i9x/wheels/d9/1a/40/50a35a078ec5cbb53d17f48a4d94ce402a94697350fb348ce5
Successfully built exactcover


Pentomino Puzzle example:<br/>
See: [Exact Cover I](https://garethrees.org/2015/11/09/exact-cover/)

In [3]:
import exactcover
import pprint

# The twelve pentominos, along with their names.
pentominos = {
    'f': [(1, 0), (2, 0), (0, 1), (1, 1), (1, 2)],
    'i': [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)],
    'l': [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3)],
    'n': [(1, 0), (1, 1), (0, 2), (1, 2), (0, 3)],
    'p': [(0, 0), (1, 0), (0, 1), (1, 1), (0, 2)],
    't': [(0, 0), (1, 0), (2, 0), (1, 1), (1, 2)],
    'u': [(0, 0), (2, 0), (0, 1), (1, 1), (2, 1)],
    'v': [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)],
    'w': [(0, 0), (0, 1), (1, 1), (1, 2), (2, 2)],
    'x': [(1, 0), (0, 1), (1, 1), (2, 1), (1, 2)],
    'y': [(1, 0), (0, 1), (1, 1), (1, 2), (1, 3)],
    'z': [(0, 0), (1, 0), (1, 1), (1, 2), (2, 2)],
}

def rotations(shape):
    """List the rotations and reflections of a pentomino."""
    # The 8 orientations are combinations of 3 primitive operations, x-flip,
    # y-flip, and transpose.
    def xflip(shape):
        return [(4 - x, y) for x, y in shape]
    def yflip(shape):
        return [(x, 4 - y) for x, y in shape]
    def transpose(shape):
        return [(y, x) for x, y in shape]
    def align(shape):
        mx = min(x for x, y in shape)
        my = min(y for x, y in shape)
        return sorted((x - mx, y - my) for x, y in shape)

    # Helper to add unique shapes to a list
    out = []
    def add(shape):
        rotation = align(shape)
        if rotation not in out:
            out.append(rotation)

    add(shape)
    add(transpose(xflip(shape)))
    add(xflip(yflip(shape)))
    add(transpose(yflip(shape)))
    add(xflip(shape))
    add(yflip(shape))
    add(transpose(shape))
    add(transpose(xflip(yflip(shape))))
    return out


def positions(shape, width, height, world):
    """List the positions a pentomino can be in.
    'world' is a set containing all of the legal squares on the board.
    'width' and 'height' are the maximum extents of world.
    """
    for y in range(height):
        for x in range(width):
            new_shape = [(x + xx, y + yy) for xx, yy in shape]
            if set(new_shape).issubset(world):
                yield new_shape


def board():
    """List the legal squares on the board.
    The board is a standard 8x8 chess board with the center 4 squares removed.
    """
    b = set((x, y) for x in range(8) for y in range(8)
            if not (3 <= x < 5 and 3 <= y < 5))
    return b


def matrix():
    """Compute the set of covers representing the problem."""
    b = board()

    covers = []
    for name, shape in pentominos.items():
        for rotation in rotations(shape):
            for position in positions(rotation, 8, 8, b):
                covers.append([name] + sorted(position))
            # Choose l as anchor to obtain unique solutions
            if name == 'l':
                break
    return covers


def solution_str(solution):
    """Turn a covering into a string picture representation."""
    grid = [[' ' for i in range(8)] for j in range(8)]

    # Mark unoccupied squares.
    for x, y in board():
        grid[y][x] = '.'

    # Mark each pentomino by its name.
    for row in solution:
        c = row[0]
        for x, y in row[1:]:
            grid[y][x] = c

    return "\n".join(''.join(row) for row in grid)


def main():
    m = matrix()

    print("Example covering:")
    # Take the first result from the iterator.
    solution = next(exactcover.Coverings(m))
    pprint.pprint(solution)
    print()
    print(solution_str(solution))
    print()

    # Count the number of results returned by the iterator.
    print('There are',len(list(exactcover.Coverings(m))),'unique tilings.')

if __name__ == '__main__':
    main()

Example covering:
(['l', (0, 0), (0, 1), (0, 2), (0, 3), (1, 3)],
 ['x', (3, 1), (4, 0), (4, 1), (4, 2), (5, 1)],
 ['p', (1, 0), (1, 1), (2, 0), (2, 1), (3, 0)],
 ['t', (1, 2), (2, 2), (2, 3), (2, 4), (3, 2)],
 ['v', (5, 0), (6, 0), (7, 0), (7, 1), (7, 2)],
 ['f', (5, 2), (6, 1), (6, 2), (6, 3), (7, 3)],
 ['z', (5, 3), (5, 4), (6, 4), (7, 4), (7, 5)],
 ['u', (0, 4), (0, 5), (0, 6), (1, 4), (1, 6)],
 ['i', (0, 7), (1, 7), (2, 7), (3, 7), (4, 7)],
 ['y', (1, 5), (2, 5), (2, 6), (3, 5), (4, 5)],
 ['w', (5, 5), (6, 5), (6, 6), (7, 6), (7, 7)],
 ['n', (3, 6), (4, 6), (5, 6), (5, 7), (6, 7)])

lpppxvvv
lppxxxfv
ltttxffv
llt  zff
uut  zzz
uyyyywwz
uuynnnww
iiiiinnw

There are 65 unique tilings.


In [4]:
m=matrix()
%timeit for _ in exactcover.Coverings(m): pass

1 loop, best of 3: 366 ms per loop


8 Queens [Puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle)

In [14]:
import exactcover

def print_solution(n, solution):
    a = [
        [ "." for col in range(n) ]
        for row in range(n)
    ]
    for entry in solution:
        row = int(entry[0][1:])
        col = int(entry[1][1:])
        a[row][col] = "*"

    for row in a:
        print("".join(row))

n = 8
matrix = []
for row in range(n):
    for col in range(n):
        matrix.append((
            "r%d" % row,
            "c%d" % col,
            "d%d" % (row + col),
            "a%d" % (row + n-1 - col)
        ))

# The diagonal and antidiagonal constraints are secondary,
# since not every diagonal will have a queen on it.
secondary = set(( "d%d" % i for i in range(2*n-1) ))
secondary.update(set(( "a%d" % i for i in range(2*n-1) )))
count=0
for solution in exactcover.Coverings(matrix, secondary):
    count+=1
    print(str(count)+':')
    print_solution(n, solution)
print('Total solutions',count)


{'a10', 'd13', 'd8', 'd10', 'd12', 'a4', 'a0', 'd7', 'a6', 'a12', 'a8', 'd4', 'a7', 'a3', 'a11', 'd9', 'a1', 'd0', 'd1', 'a13', 'd2', 'a9', 'a2', 'a14', 'd3', 'd14', 'a5', 'd5', 'd11', 'd6'}


NameError: ignored

In [0]:
%timeit for _ in exactcover.Coverings(matrix, secondary): pass

1000 loops, best of 3: 374 µs per loop


Example from [Wikipedia](https://en.wikipedia.org/wiki/Knuth%27s_Algorithm_X)

In [16]:
m=[ [ 'A', 1, 4, 7 ],
    [ 'B', 1, 4 ],
    [ 'C', 4, 5, 7 ],
    [ 'D', 3, 5, 6 ],
    [ 'E', 2, 3, 6, 7 ],
    [ 'F', 2, 7 ]]
secondary=set(['A','B','C','D','E','F'])
for solution in exactcover.Coverings(m,secondary):
    print(solution)

(['B', 1, 4], ['D', 3, 5, 6], ['F', 2, 7])
